# Bases de datos 

__No va a preguntar de Bases de Datos porque ya tenemos una asignatura para eso.__

Las bases de datos nos pueden servir como fuente, como destino de los resultados, o ambas. Vamos a ver cómo leer y escribir datos desde Python.

Para estos ejemplos, utilizaremos una base de datos `sqlite` en local.

In [1]:
import sqlite3 # Cada base de datos tiene su librería de pandas. En este caso usamos la de SQLite.
import pandas as pd

## Conexión

Esto varía dependiendo de la BD a la que te vayas a conectar:

* sqlite: es neceario el módulo `sqlite3`, que vamos a utilizar
* PostgreSQL: necesitas el módulo `psycopg2`
* MySQL: hay varias alternativas, como `pymysql` o `mysqlclient`

In [2]:
conn = sqlite3.connect('11_bd_temporal.sqlite') # Con esta línea creamos una DB vacía (porque no hay ninguna) y nos
# conectamos a ella. 

# Nos lo crea en el notebook porque no le hemos especificado una ruta.

## Escritura

Podemos volcar un dataframe a una tabla de la base de datos con [`DataFrame.to_sql`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html).

In [4]:
alquiler = pd.read_csv('dat/alquiler-madrid-distritos.csv')
alquiler.head()

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


In [7]:
alquiler.to_sql('alquiler', conn) # Creamos una tabla llamada 'alquiler' en la DB creada a la que nos hemos conectado con conn.

## Consultas

Para lanzar consultas a la base de datos y obtener el resultado en un dataframe, podemos usar [`pd.read_sql()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html)

In [8]:
query = ''' 
select distrito, ano, quarter, precio
from alquiler
where distrito = 'Retiro'
and ano between 2012 and 2014
'''

# La triple comilla no es necesaria, pero hace que las cosas se vean mejor cuando escribimos la query en el script.

pd.read_sql(query, conn) # Leemos la DB y obtenemos la tabla que se obtiene al aplicar la query.

,distrito,ano,quarter,precio
0,Retiro,2012,1,12.601891
1,Retiro,2012,2,12.363832
2,Retiro,2012,3,12.167435
3,Retiro,2012,4,11.951317
4,Retiro,2013,1,11.431686
5,Retiro,2013,2,11.419576
6,Retiro,2013,3,11.350000
7,Retiro,2013,4,11.519675
8,Retiro,2014,1,11.632763
9,Retiro,2014,2,11.400000


#### Ejercicio

Parametriza la query, de forma que el distrito, el año mínimo y el año máximo sean variables. Invoca la query utilizando el argumento `params` de `read_sql`

Parametrizar es importante para evitar inyección de SQL. Esto es especialmente importante si los datos son introducidos por usuarios. [xkcd](http://imgs.xkcd.com/comics/exploits_of_a_mom.png)

## Carga de resultados

Aunque hemos visto que se puede crear y alimentar una tabla en el momento con `to_sql`, lo más habitual es que nuestra base de datos ya tenga las tablas creadas (con su esquema bien definido, índices, claves foráneas, ...) y que queramos añadir datos.

Vamos a crear una tabla definiendo su esquema, para ver como podríamos añadir los datos de un dataframe sin crearla de cero en `to_sql`.

In [13]:
c = conn.cursor()
c.execute('''create table alquiler_2
             (distrito text, ano integer, quarter integer, precio real)''') # Para modificar tablas se usa execute, no query.

OperationalError: table alquiler_2 already exists

Comprobamos que está vacía

In [10]:
pd.read_sql('select * from alquiler_2', conn)

,distrito,ano,quarter,precio


Agregamos los datos a la tabla existente

In [11]:
alquiler.to_sql('alquiler_2', conn, index=False, if_exists='append')

In [12]:
pd.read_sql('select * from alquiler_2 limit 5', conn) 

,distrito,ano,quarter,precio
0,Arganzuela,2007,2,13.066587
1,Barajas,2007,2,11.199855
2,Carabanchel,2007,2,11.127661
3,Centro,2007,2,17.746404
4,Chamartín,2007,2,14.386480


## Cierre de conexión

In [14]:
conn.close() # Mejor cerrar la conexión después de acabar de trabajar. Aunque no sea obligatoria.

#### Ejercicio

Crea una BD de sqlite nueva donde cargues información de alojamientos y barrios de Madrid de AirBnB. La información está disponible [aquí](http://insideairbnb.com/get-the-data.html). Los ficheros a cargar son `listings.csv` y `neighbourhoods.csv`.

Una vez cargados, prueba a hacer algunas consultas que sean de tu interés.